**Warning: Offensive language**

In [1]:
from ml_pipeline import experiment

/home/rutger/miniconda3/envs/pynlp/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
2022-09-25 09:12:17.402407: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-09-25 09:12:17.402446: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
res = experiment.run(
    task_name='vua_format',
    data_dir='../pynlp/data/dataset1/',
    pipeline_name='naive_bayes_counts_lex',
    print_predictions=False
)

>> Running vua_format experiment
>> Loading data...
>> retrieving train/data instances...
>> training pipeline naive_bayes_counts_lex
>> testing...
>> evaluation...
              precision    recall  f1-score   support

         NOT       0.80      0.62      0.70      1972
         OFF       0.62      0.81      0.70      1539

    accuracy                           0.70      3511
   macro avg       0.71      0.71      0.70      3511
weighted avg       0.72      0.70      0.70      3511



In [3]:
import pandas as pd

In [4]:
df_gibert_test = pd.read_csv('../pynlp/data/gibert/testData.csv', delimiter='\t')

In [5]:
res

Pipeline(steps=[('prep',
                 <ml_pipeline.preprocessing.Preprocessor object at 0x7fe7bc046490>),
                ('frm', CountVectorizer()), ('clf', MultinomialNB())])

In [6]:
prep = res['prep']
frm = res['frm']
clf = res['clf']

In [49]:
x = df_gibert_test['Text']

In [52]:
res = prep.transform(x)

print('Before processing:')
print(x[1])
print('\n')
print('After processing:')
print(res[1])

Before processing:
I know exactly what you 're saying , I just hate to see those worthless apes benefit from the great things that whites do .


After processing:
I know exactly what you ' re saying , I just hate to see those worthless hate apes benefit from the great things that whites do .


With a lexicon `hate` is prepended to any token that is found in the lexicon. In this case `apes` is in the lexicon:
```
Entry;Pos;Label
...
apes;anypos;hate
...
```
hence \
```...hate to see those worthless apes benefit...``` \
is transformed to  \
```...hate to see those worthless hate apes benefit...```